# <center> Learning of structured data </center>
### <center> Date: 15.01.2023 </center>

Authors: 
- Jochen Schmidt
- Julian Tilly 
- Lennard Rose
- Marius Benkert
- Christopher Neeb



This portfolio is based on a Kaggle Competition, which can be found [here](https://www.kaggle.com/competitions/learning-of-structured-datafhws-ws2223 "Kaggle Competition"). It is a skeleton data time series classification.

### Exercise 1: Data Set Exploration and Visualization

- Load the time series movement sequence data set in the .csv format. The sequences are time recordings of movements (joint angles) with different lengths. Labels are given in the file names.
- Try to visualize the data to get a first idea

#### Data Exploration

##### Pose Visualization

We began from the image of the body_25-format provided in the kaggle-competition.


<img src="./images/keypoints_pose_25.png" width="300"/>

We implemented a way to visualize the data as graphs.
We took the joint positions coordinates as nodes and connected them with edges according to the image of the body_25-format.
After generating one of these graphs for every row of the .csv data, we created a gif of all of the files poses.

<img src="./images/violin/violinning.gif" width="500"/>

The gif shows the sequence of one of the training csvs. The pose that is being carried out is someone playing the violin.
For visualization the image has been flipped, all (at least all we know) poses were upside down. Which was of no concern for the training.
The test data looked similar, so no big surprises or traps here.

The first thing we noticed was the missing legs. We checked the distributions of specific datapoints to see if bodyparts are missing.

##### Data distribution

<img src="./images/distribution_overall_hbar.png" width="500"/>

These percentages are of the left armpit angle (for the angles), left knee (for the legs) and left heel (for the feet).
We didn´t checked for all joints because we just needed a first impression. With so many values missing, we decided we had to add the angles by ourselves (as seen in the Additional Angle Calculation part) and to leave out everything below the hip, because information was too sparse.

<img src="./images/distribution_bars.png" width="700"/>

The different activities were distributed almost equally with bigger differences in the lower body joint coordinates.

The 3 principal components explained ~57%, ~10% and ~8% of the variances for a total of almost 75%!

<img src="./images/pca2.png" width="500"/>

The 2D plot didn´t give any additional insights.

<img src="./images/pca3.png" width="500"/>

The 3D plot didn´t provided any additional insights either.

Both plots consisted of wildly confused datapoints. With patterns that looked like scribbled lines.

<img src="./images/tsne2_10k.png" width="500"/>

The visualization of TSNE on 10000 frames of the data looked more like clusters.

<img src="./images/tsne2_100k.png" width="500"/>

We used PCA as long as we are around 99% explained variance to reduce the dimensions for the TSNE. With 16 dimensions left, we tried TSNE with 2 components on 100000 frames (out of >600000) to see if it gives us more differentiable cluster. The visualized result had far more overlapping clusters than the 10k TSNE.

<img src="./images/tsne3.png" width="500"/>

The 3D plot of a TSNE with 3 components on the 100k frames did not showed anything different from the 2 component one above.

The complete data exploration and visualization can be found in Data_Exploration.ipynb including code and additional poses

### Exercise 2: Embedding and Modeling

- Select a technique how to represent / process the given data - this could be an explicit or implicit data representation or also a simple statistic like histograms, bit-encodings or alike
- Process the data to obtain a meaning and powerful representation
- Create a process to obtain a classification model for the given data. This should include meta parameter optimization.
- Provide a brief explanation of the algorithm-design you used

#### Techniques and Representations

##### Averages Calculation

As kind of a baseline model, we thought it would be interesting to use just some simple statistics of the data. For that, we calculated the average of each joint angle over the whole sequence. So for every training sample (one .csv), we only had one feature vector, that is the average of each joint angle over the whole sequence (12 features).

With just using the averages as features, we got an accuracy score on the public test data from kaggle of 0.61. As a model we used the *lazypredict* library in order to get a first overview of the performance of different classifiers. We used the default parameters for each classifier.

This is not a very good score, but it is a good baseline to compare our other models to.

##### Additional Angle Calculation

<img src="./images/angles_example.png" alt="Additional Angles" width="500"/>

The dataset contained some of the angles between the armpit and the elbow already, we added the missing ones. We additionally calculated angles for the head, shoulder and hip as seen in the picture above. We also added the distance between the right and left eye, right and left shoulder, as well as the right and left hip to account for rotation of the body.
With this angles we reduced the theoretically necessary features from 48 to 8.
This should help in differntiating between the different poses.
For example, in sequences where someone is playing violin the head is tilted more to the left.
We left the angles of the lower body out, because there were too little information provided (see data exploration above).
The additional features were intended to be used along with the other features as well as alone.


##### Sliding Windows

In a first step, we concatenated all the training/test csv files to one separate big sequence, which resulted in sequences of dimensions:

- **Training**: (653422, 80) 
- **Test**: (207308, 80)

In a second step, we used sliding windows over the single complete sequences. Typically we set the window size to $100$ and the step size to $50$. The label for the window was set to the most frequent label in the window itself. This yielded training/test data of dimensions:

- **Training**: (13067, 100, 79), 
- **Test**: (4145, 100, 79),

where the first dimension is the number of windows, the second dimension is the window size and the third dimension is the number of features.

This way we had fixed length sequences, which are needed for the various models that were used.

#### Algorithms


##### k-Nearest Neighbor with Dynamic Time Warping

While doing research for suiting timeseries frameworks in python, we stumbled upon **tslearn** which works with the sklearn schemata. This framework has various classifiers optimized for time series datasets such as kNN and SVM and metrics. As we came across the Dynamic Time Warping metric we had to try it out and were surprised, with a train/validation split the classifier had a accuracy score of *96%* but could not transfer this performance to the test data.

##### Ensemble Predictor

As a final idea and step, we wanted to combine the predictions already made by the different models into an ensemble classifier. As a lightweight solution, we simply used the already submitted .csv-files and applied an equally weighted voting strategy. Despite combining very good results, no improvement could be achieved.

#### Neural Networks

##### Training
- 5 fold crossvalidation
- Reduce learning rate on plateau
- Early stopping
  
##### Models
- Multilayer perceptron (MLP)
- Gatet recurrent unit (GRU)
- Convolutional neural network (CNN)
  - CNN 1D
  - CNN 2D
- Residual neural network (ResNet)
  - ResNet 18
  - ResNet 34
- Convolutional long short-term memory (ConvLSTM)

#### Neural Networks and Hyperparameter optimization

A hyperparametric optimization was performed on three models (MLP, GRU, CNN).
The following parameters and values were tested:
- batch size: 64, 128, 256
- optimizer: adam, sgd
- learning rate factor: 0.1, 0.5, 0.7 (factor by which the learning rate will be reduced. new_lr = lr * factor)
- learning rate patience: 5, 10, 15 (number of epochs with no improvement after which learning rate will be reduced)
- early stopping patience: 10, 50, 70 (number of epochs with no improvement after which training will be stopped)

One model was trained for each possible combination, which corresponds to 162 models each for MLP, GRU, CNN. For the evaluation of the best parameters, the validation accuracy was used.

The best parameters are listed in the following table.
| Model  | batch size  | optimizer  | learning rate factor  | learning rate patience  | early stopping patience | Training Runtime [hh:mm] |
|--------|-------------|------------|-----------------------|-------------------------|-------------------------|--------------------------|
| **MLP** | 256  | adam  | 0.5  | 10  | 50  | 09:22  |
| **GRU** | 256  | adam  | 0.7  | 15  | 50  | 24:15  |
| **CNN** | 64   | adam  | 0.5  | 5   | 70  | 18:51  |

### Exercise 3: Evaluation

- Estimate theoretical and practical the memory and runtime complexity of your approach - make a distinction between training and test. Address also the sparsity of your model.
- Evaluate the resulting model by categorization accuracy.

#### Complete Overview of Results

If 5 values are entered in the columns Epochs and Accuracy, then a 5 Fold Cross-validation was performed. The Accuracy column then contains the validation accuracy. If only one value is entered, then training was performed on the entire data and the training accuracy was entered. The training runtime was achieved using one GPU. 


| Model  | Features  | Epochs  | Total params  | Trainable params  | Training Runtime [hh:mm] | Accuracy [%]  | Kaggle Public score [%] |
|--------|-----------|---------|--------------:|------------------:|--------------------------|---------------|-------------------------|
| **MLP**     | 79  | 97, 95, 92, 116, 105  | 2,057,733  | 2,056,965  | 00:11  | 98.09, 97.63, 97.21, 97.59, 96.82  | 67.605 |
| **CNN 1D**  | 12  | 500 | 326,933 | 325,781 | 00:06 | 96.82 | 76.99
| **CNN 2D**  | 79  | 90, 91, 93, 77, 101  | 30,542,277  | 30,541,637  | 00:36  | 98.78, 98.55, 98.09, 98.24, 98.51  | 81.69  |
| **CNN 2D**  | 79  | 259  | 30,542,277  | 30,541,637  | 00:23  | 100.0  | 84.037  |
| **CNN 2D**  | 79  | 166  | 30,542,277  | 30,541,637  | 00:18  | 99.97  | 86.384  |
| **CNN 2D**  | 79  | 100  | 30,542,277  | 30,541,637  | 00:12  | 99.98  | **89.671**  |
| **CNN 2D**  | 87  | 100  | 33,753,541  | 33,752,901  | 00:13  | 99.98  | 86.854  |
| **CNN 2D**  | 12  | 236  | 4,049,349  | 4,048,709  | 00:13  | 99.95  | 73.239  |
| **CNN 2D V2**  | 87  | 237  | 3,772,101  | 3,770,949  | 00:19  | 99.98  | 79.342  |
| **CNN 2D V2**  | 79  | 100  | 3,411,653  | 3,410,501  | 00:08  | 99.98  | 81.22  |
| **GRU**     | 79  | 80, 72, 66, 67, 64  | 4,291,845  | 4,290,053  | 00:19  | 99.23, 98.78, 98.81, 98.81, 98.93  | 76.056  |
| **GRU**     | 79  | 126  | 4,291,845  | 4,290,053  | 00:08  | 99.99  | 78.403   |
| **Conv LSTM**   | 79  | 63, 59, 57, 60, 60  | 26,911,493  | 26,910,469  | 07:07  | 98.85, 98.74, 98.09, 98.58, 98.35  | 76.995  |
| **Conv LSTM**   | 79  | 261  | 26,911,493  | 26,910,469  | 07:08  | 100.0  | 81.22  |
| **Conv LSTM**   | 79  | 100  | 26,911,493  | 26,910,469  | 02:45  | 100.0  | 81.69  |
| **Conv LSTM**   | 87  | 190  | 29,532,933  | 29,531,909  | 05:37  | 100.0  | 81.69  |
| **ResNet 18**   | 79  | 71, 64, 70, 68, 64  | 11,184,453  | 11,173,829  | 00:22  | 99.46, 99.08, 98.93, 99.0, 98.89  | 74.647  |
| **ResNet 34**   | 79  | 77, 82, 69, 65, 67  | 21,300,037  | 21,281,989  | 00:36  | 99.23, 98.97, 98.78, 98.97, 98.97  | 77.464  |
| **ResNet 34**   | 79  | 305  | 21,300,037  | 21,281,989  | 00:33  | 100.0  | 66.666  |
| **Linear SVC**  | 1580  | -  | -  | -  | -  | -  | 28.638  |
| **Linear SVC**  | 3160  | -  | -  | -  | -  | -  | 23.004  |
| **Logistic Regression**  | 3160  | -  | -  | -  | -  | -  | 39.906  |

Our first attempt, an MLP has the largest discrepancy between training and test accuracy. There are a few more thoughts on this below. Otherwise, it is good in terms of training time and memory requirements, but this is of little use due to the mediocre results. 
 
 A second approach using a CNN gave the best results. Although the resulting model has the largest memory cost, it was quick to train and gave the best results on the test data set and finally in the Kaggle Competition. 
 
 Another model used LSTM. Only slightly less data was trained than with the CNN, so the memory requirement is also very high here. The results on the test data set are ok but worse than on the CNN. In particular, the training duration of several hours is particularly significant here compared to other methods used. 
 
 A GRU model was used as a counterpart to the lstm. the memory requirement is only one sixth of what was needed for the lstm. The accuracy is reduced by two to five percentage points, but this is the fastest method. 
 
 Last but not least, the RNN should also be mentioned, which is theoretically optimally suited for time series analyses. In practice, however, it had an immense memory requirement and a mediocre training duration for the worst results in a direct comparison. 
 
 In general, the discrepancy between training and test set is striking. With results like those of the MLP, the assumption would be that there is an overfitting, because the training accuracy is very high for all models used. However, since there is also a not insignificant gap between our own tests and those of the Kaggle Competition in all models, the assumption is that the data in Kaggle are of a different nature in some way. 
 


#### Visual Results

<img src="./Results/CNN/Confusion_matrix_Fold_0.png" width="700"/>

Confusion matrix of the best performing CNN model.

<img src="./Results/CNN/Loss_and_Accuracy_Fold_0.png" width="600"/>

Loss and accuracy of the best performing CNN model.

<img src="./Results/Total_params_and_Accuracy.png" width="700"/>

Graphic describing the number of parameters used in the models compared to the Kaggle Public Scoring.

<img src="./Results/Ratio_and_Accuracy.png" width="700"/>

Graphic describing the ratio between the number of parameters used in the models compared to their prediction accuracy.

More parameters does not mean better accuracy.